This demo introduces **Ray tune's** key concepts using a trivial examples. This example is derived from [Ray Tune basic example](https://docs.ray.io/en/latest/tune/examples/tune_basic_example.html). Basically, there are three basic steps or Ray Tune pattern for you as a newcomer to get started with using Ray Tune.

 1. Setup your config space and define your trainable and objective function
 2. Use tune to execute your training, supplying the appropriate arguments including: search space, [search algorithms](https://docs.ray.io/en/latest/tune/api_docs/suggestion.html#blendsearch) or [trial schedulers](https://docs.ray.io/en/latest/tune/api_docs/schedulers.html#tune-schedulers)
 3. Examine analyse the results
 
 ![](https://docs.ray.io/en/latest/_images/tune-workflow.png)


See also the [Hyperparameter Tuning References](References-Hyperparameter-Tuning.ipynb) notebook and the [Tune documentation](http://tune.io), in particular, the [API reference](https://docs.ray.io/en/latest/tune/api_docs/overview.html). 


In [1]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

from ray import tune

In [2]:
NUM_OF_ACTORS = 4           # degree of parallel trials; each actor will have a separate trial
NUM_OF_CPUS_PER_ACTOR = 1   # number of CPUs per actor

In [3]:
ray_params = RayParams(num_actors=NUM_OF_ACTORS, cpus_per_actor=NUM_OF_CPUS_PER_ACTOR)

## Step 1: Define our training function to use with Ray Tune `ray.tune(...)`

In [4]:
def train_model(config:dict):
    # create the dataset
    train_X, train_y = load_breast_cancer(return_X_y=True)
    # Convert to RayDMatrix data structure
    train_set = RayDMatrix(train_X, train_y)

    # Empty dictionary for the evaluation results reported back
    # to tune
    evals_result = {}

    # Train the model with XGBoost train
    bst = train(
        params=config,                       # our hyperparameter search space
        dtrain=train_set,                    # our RayDMatrix data structure
        evals_result=evals_result,           # place holder for results
        evals=[(train_set, "train")],
        verbose_eval=False,
        ray_params=ray_params)                # distributed parameters configs for Ray Tune

    bst.save_model("model.xgb")

## Step 2: Define our hyperparameter search space

In [6]:
 # Specify the hyperparameter search space
config = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "eta": tune.loguniform(1e-4, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "max_depth": tune.randint(1, 9)
}

## Step 3: Run Ray tune main trainer and examine the results

In [7]:
# Run tune
analysis = tune.run(
    train_model,
    config=config,
    metric="train-error",
    mode="min",
    num_samples=4,
    
    resources_per_trial=ray_params.get_tune_resources()
)

2022-01-04 14:57:14,805	INFO services.py:1338 -- View the Ray dashboard at http://127.0.0.1:8265
2022-01-04 14:57:16,121	WARNING function_runner.py:561 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,eta,max_depth,subsample
train_model_a973f_00000,PENDING,,0.00122545,1,0.942262
train_model_a973f_00001,PENDING,,0.000668487,7,0.667592
train_model_a973f_00002,PENDING,,0.00256656,8,0.626182
train_model_a973f_00003,PENDING,,0.00070348,3,0.560148


(ImplicitFunc pid=83544) 2022-01-04 14:57:18,435	INFO main.py:976 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=83542) 2022-01-04 14:57:18,411	INFO main.py:976 -- [RayXGBoost] Created 4 new actors (4 total actors). Waiting until actors are ready for training.
(ImplicitFunc pid=83542) 2022-01-04 14:57:20,393	INFO main.py:1021 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=83543) [14:57:20] task [xgboost.ray]:140376179703472 got new rank 3
(_RemoteRayXGBoostActor pid=83535) [14:57:20] task [xgboost.ray]:140527845572224 got new rank 2
(_RemoteRayXGBoostActor pid=83538) [14:57:20] task [xgboost.ray]:140486506315440 got new rank 0
(_RemoteRayXGBoostActor pid=83545) [14:57:20] task [xgboost.ray]:140664214716080 got new rank 1
(ImplicitFunc pid=83544) 2022-01-04 14:57:21,009	INFO main.py:1021 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=83534) [14:57:21] task [xgboost.ray]

Result for train_model_a973f_00000:
  date: 2022-01-04_14-57-21
  done: false
  experiment_id: f8f5f33c48f14891b87d55160334efee
  hostname: Juless-MacBook-Pro-16-inch-2019
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 83542
  time_since_restore: 3.231962203979492
  time_this_iter_s: 3.231962203979492
  time_total_s: 3.231962203979492
  timestamp: 1641337041
  timesteps_since_restore: 0
  train-error: 0.079086
  train-logloss: 0.692296
  training_iteration: 1
  trial_id: a973f_00000
  
Result for train_model_a973f_00000:
  date: 2022-01-04_14-57-21
  done: true
  experiment_id: f8f5f33c48f14891b87d55160334efee
  experiment_tag: 0_eta=0.0012254,max_depth=1,subsample=0.94226
  hostname: Juless-MacBook-Pro-16-inch-2019
  iterations_since_restore: 10
  node_ip: 127.0.0.1
  pid: 83542
  time_since_restore: 3.2896409034729004
  time_this_iter_s: 0.004565715789794922
  time_total_s: 3.2896409034729004
  timestamp: 1641337041
  timesteps_since_restore: 0
  train-error: 0.079086
  tr

(ImplicitFunc pid=83542) 2022-01-04 14:57:21,696	INFO main.py:1500 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 3.32 seconds (1.30 pure XGBoost training time).


...

Result for train_model_a973f_00002:
  date: 2022-01-04_14-57-27
  done: true
  experiment_id: ceb9bc0ebc86423282de1e849818c098
  experiment_tag: 2_eta=0.0025666,max_depth=8,subsample=0.62618
  hostname: Juless-MacBook-Pro-16-inch-2019
  iterations_since_restore: 10
  node_ip: 127.0.0.1
  pid: 83615
  time_since_restore: 3.7706780433654785
  time_this_iter_s: 0.006224870681762695
  time_total_s: 3.7706780433654785
  timestamp: 1641337047
  timesteps_since_restore: 0
  train-error: 0.024605
  train-logloss: 0.671759
  training_iteration: 10
  trial_id: a973f_00002
  


(ImplicitFunc pid=83615) 2022-01-04 14:57:27,504	INFO main.py:1500 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 3.77 seconds (1.53 pure XGBoost training time).
(ImplicitFunc pid=83623) 2022-01-04 14:57:28,396	INFO main.py:1021 -- [RayXGBoost] Starting XGBoost training.
(_RemoteRayXGBoostActor pid=83637) [14:57:28] task [xgboost.ray]:140433091624528 got new rank 3
(_RemoteRayXGBoostActor pid=83636) [14:57:28] task [xgboost.ray]:140242234867280 got new rank 2
(_RemoteRayXGBoostActor pid=83634) [14:57:28] task [xgboost.ray]:140513892404816 got new rank 0
(_RemoteRayXGBoostActor pid=83635) [14:57:28] task [xgboost.ray]:140434031050320 got new rank 1


Result for train_model_a973f_00003:
  date: 2022-01-04_14-57-29
  done: false
  experiment_id: 7e1779e7e647487ca5dfcf287f650dfd
  hostname: Juless-MacBook-Pro-16-inch-2019
  iterations_since_restore: 1
  node_ip: 127.0.0.1
  pid: 83623
  time_since_restore: 3.4027671813964844
  time_this_iter_s: 3.4027671813964844
  time_total_s: 3.4027671813964844
  timestamp: 1641337049
  timesteps_since_restore: 0
  train-error: 0.047452
  train-logloss: 0.692544
  training_iteration: 1
  trial_id: a973f_00003
  
Result for train_model_a973f_00003:
  date: 2022-01-04_14-57-29
  done: true
  experiment_id: 7e1779e7e647487ca5dfcf287f650dfd
  experiment_tag: 3_eta=0.00070348,max_depth=3,subsample=0.56015
  hostname: Juless-MacBook-Pro-16-inch-2019
  iterations_since_restore: 10
  node_ip: 127.0.0.1
  pid: 83623
  time_since_restore: 3.447298049926758
  time_this_iter_s: 0.004242897033691406
  time_total_s: 3.447298049926758
  timestamp: 1641337049
  timesteps_since_restore: 0
  train-error: 0.033392
  

Trial name,status,loc,eta,max_depth,subsample,iter,total time (s),train-logloss,train-error
train_model_a973f_00000,TERMINATED,127.0.0.1:83542,0.00122545,1,0.942262,10,3.28964,0.684677,0.079086
train_model_a973f_00001,TERMINATED,127.0.0.1:83544,0.000668487,7,0.667592,10,5.42616,0.687487,0.022847
train_model_a973f_00002,TERMINATED,127.0.0.1:83615,0.00256656,8,0.626182,10,3.77068,0.671759,0.024605
train_model_a973f_00003,TERMINATED,127.0.0.1:83623,0.00070348,3,0.560148,10,3.4473,0.687265,0.033392


(ImplicitFunc pid=83623) 2022-01-04 14:57:29,649	INFO main.py:1500 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 3.45 seconds (1.25 pure XGBoost training time).
2022-01-04 14:57:29,782	INFO tune.py:626 -- Total run time: 13.66 seconds (12.91 seconds for the tuning loop).


In [8]:
print("Best hyperparameters", analysis.best_config)

Best hyperparameters {'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.0006684866042847465, 'subsample': 0.6675916933432418, 'max_depth': 7}


In [9]:
analysis.results_df.head(5)

/usr/local/anaconda3/lib/python3.8/site-packages/ray/tune/analysis/experiment_analysis.py:262: UserWarning: Dataframes will use '/' instead of '.' to delimit nested result keys in future versions of Ray. For forward compatibility, set the environment variable TUNE_RESULT_DELIM='/'
  warnings.warn(


,train-logloss,train-error,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,experiment_id,date,timestamp,...,iterations_since_restore,experiment_tag,config.tree_method,config.objective,config.eval_metric,config.eta,config.subsample,config.max_depth,config.nthread,config.n_jobs
trial_id,,,,,,,,,,,,,,,,,,,,,
a973f_00000,0.684677,0.079086,0.004566,True,None,None,10,f8f5f33c48f14891b87d55160334efee,2022-01-04_14-57-21,1641337041,...,10,"0_eta=0.0012254,max_depth=1,subsample=0.94226",approx,binary:logistic,"[logloss, error]",0.001225,0.942262,1,1,1
a973f_00001,0.687487,0.022847,0.009679,True,None,None,10,861f8023db0340fa9f5942d13e1aba4b,2022-01-04_14-57-23,1641337043,...,10,"1_eta=0.00066849,max_depth=7,subsample=0.66759",approx,binary:logistic,"[logloss, error]",0.000668,0.667592,7,1,1
a973f_00002,0.671759,0.024605,0.006225,True,None,None,10,ceb9bc0ebc86423282de1e849818c098,2022-01-04_14-57-27,1641337047,...,10,"2_eta=0.0025666,max_depth=8,subsample=0.62618",approx,binary:logistic,"[logloss, error]",0.002567,0.626182,8,1,1
a973f_00003,0.687265,0.033392,0.004243,True,None,None,10,7e1779e7e647487ca5dfcf287f650dfd,2022-01-04_14-57-29,1641337049,...,10,"3_eta=0.00070348,max_depth=3,subsample=0.56015",approx,binary:logistic,"[logloss, error]",0.000703,0.560148,3,1,1
